In [29]:
# 경훈님 데이터 변환작업 xml -> txt 및 클래스 변환

import os
import xml.etree.ElementTree as ET
import glob
from tqdm import tqdm


def xml_to_yolo_bbox(bbox, w, h):
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2) / h
    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h
    return [x_center, y_center, width, height]

def parsing(annot_path, label_path):
    classes = ["Text_Table2", "Text_Table1", "Info", "Table"]

    files = glob.glob(os.path.join(annot_path, '*.xml'))
    for fil in tqdm(files):
        basename = os.path.basename(fil)
        filename = os.path.splitext(basename)[0]
        result = []

        tree = ET.parse(fil)
        root = tree.getroot()
        width = int(root.find("size").find("width").text)
        height = int(root.find("size").find("height").text)
        
        for obj in root.findall('object'):
            label = obj.find("name").text
            
            # 예외 처리: Table_Text2 -> Text_Table2, Table_Text1 -> Text_Table1
            if label == "Table_Text2":
                label = "Text_Table2"
            elif label == "Table_Text1":
                label = "Text_Table1"
            

            if label in classes:
                index = classes.index(label)
                pil_bbox = [int(x.text) for x in obj.find("bndbox")]
                yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)
                bbox_string = " ".join([str(x) for x in yolo_bbox])
                result.append(f"{index} {bbox_string}")
            else:
                print(f"Unknown label '{label}' found in {filename}")

        if result:
            with open(os.path.join(label_path, f"{filename}.txt"), "w", encoding="utf-8") as f:
                f.write("\n".join(result))

# 파일 경로 설정
raw_path = "/home/selectstar"
xml_path = os.path.join(raw_path, "xml_label_data")
txt_path = os.path.join(raw_path, "xml_to_txt")

# 함수 호출
parsing(xml_path, txt_path)

100%|███████████████████████████████████████| 167/167 [00:00<00:00, 3160.97it/s]


In [30]:
# 데이터 변환작업 클래스 순서 조정
import os

raw_path = "/home/selectstar"
xml_label_data_path = os.path.join(raw_path, "xml_label_data")
txt_path = os.path.join(raw_path, "xml_to_txt")

# 클래스 번호 변환 매핑
class_mapping = {0: 1, 1: 2, 2: 0, 3:3}

# 함수: 라벨링 파일 변환
def convert_labeling_file(input_file, output_file, class_mapping):
    with open(input_file, 'r') as f:
        lines = f.readlines()
    
    converted_lines = []
    for line in lines:
        parts = line.strip().split()
        if len(parts) > 0 and parts[0].isdigit():
            original_class = int(parts[0])
            if original_class in class_mapping:
                converted_class = class_mapping[original_class]
                parts[0] = str(converted_class)
                converted_lines.append(' '.join(parts) + '\n')
        else:
            converted_lines.append(line)  # 클래스 번호가 아닌 줄은 그대로 유지
    
    with open(output_file, 'w') as f:
        f.writelines(converted_lines)

# 폴더 내 라벨링 파일 변환
def convert_labeling_files_in_folder(input_folder, output_folder, class_mapping):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for filename in os.listdir(input_folder):
        if filename.endswith(".txt"):
            input_file = os.path.join(input_folder, filename)
            output_file = os.path.join(output_folder, filename)
            convert_labeling_file(input_file, output_file, class_mapping)

# 라벨링 파일 변환 수행
convert_labeling_files_in_folder(xml_label_data_path, txt_path, class_mapping)

print("변환 완료.")


변환 완료.


In [20]:
pwd

'/home/selectstar'

In [57]:
# train 전 파일 갯수 인
import os

# 경로 설정
folder_path1 = "/home/selectstar/yolov5/dataset/Table_classification/valid/images"
folder_path2 = "/home/selectstar/yolov5/dataset/Table_classification/valid/labels"
folder_path3 = "/home/selectstar/yolov5/dataset/Table_classification/train/images"
folder_path4 = "/home/selectstar/yolov5/dataset/Table_classification/train/labels"
folder_path5 = "/home/selectstar/yolov5/dataset/Table_classification/test"


# 해당 경로의 파일 개수 확인
file_count1 = len([name for name in os.listdir(folder_path1) if os.path.isfile(os.path.join(folder_path1, name))])
file_count2 = len([name for name in os.listdir(folder_path2) if os.path.isfile(os.path.join(folder_path2, name))])
file_count3 = len([name for name in os.listdir(folder_path3) if os.path.isfile(os.path.join(folder_path3, name))])
file_count4 = len([name for name in os.listdir(folder_path4) if os.path.isfile(os.path.join(folder_path4, name))])
file_count5 = len([name for name in os.listdir(folder_path5) if os.path.isfile(os.path.join(folder_path5, name))])


print(f"폴더 'Val_image'에 있는 파일 개수: {file_count1}")
print(f"폴더 'Val_labels'에 있는 파일 개수: {file_count2}")
print(f"폴더 'Train_image'에 있는 파일 개수: {file_count3}")
print(f"폴더 'Train_labels'에 있는 파일 개수: {file_count4}")
print(f"폴더 'Test_data'에 있는 파일 개수: {file_count5}")

폴더 'Val_image'에 있는 파일 개수: 67
폴더 'Val_labels'에 있는 파일 개수: 68
폴더 'Train_image'에 있는 파일 개수: 550
폴더 'Train_labels'에 있는 파일 개수: 551
폴더 'Test_data'에 있는 파일 개수: 0


In [64]:
#파일 이름 안맞는거 찾기 ㅎㅎ 
import os

folder_path1 = "/home/selectstar/yolov5/dataset/File_classification/test/images"
folder_path2 = "/home/selectstar/yolov5/runs/detect/exp"

# 해당 경로에서 파일 이름 추출 (확장자 제외)
def extract_filenames(folder_path):
    filenames = []
    for file in os.listdir(folder_path):
        if os.path.isfile(os.path.join(folder_path, file)):
            # 파일 이름에서 확장자 제외한 부분 추출
            filename = os.path.splitext(file)[0]
            filenames.append(filename)
    return filenames

# 각 경로에서 파일 이름 목록 추출
filenames1 = extract_filenames(folder_path1)
filenames2 = extract_filenames(folder_path2)

# 두 경로에서 짝이 맞지 않는 파일 찾기
missing_in_folder1 = set(filenames2) - set(filenames1)
missing_in_folder2 = set(filenames1) - set(filenames2)

print("이미지에서 누락된 파일:", missing_in_folder1)
print("라벨에서 누락된 파일:", missing_in_folder2,"\n")

이미지에서 누락된 파일: set()
라벨에서 누락된 파일: {'국민제안규정 조문별개정이유서_22_', '긴급재난지원금 사업지침(200508, 2차)_1_', '210610 2021년 자전거 이용활성화 사진 UCC 공모전 계획 v3_1_', '제7기 중분위 위촉직 민간위원 위촉계획 (1)_1_', '지방자치단체 출자ㆍ출연 기관의 운영에 관한 법률 일부개정법률안_5_', '민원서비스 종합평가 컨설팅 계획 (1)_1_', '붙임1_전자정부법 시행령 일부개정령_43_', '주민참여예산제 연찬회 계획 및 우수사례 제출 양식_5_', '지방공공기관 부채중점관리 세부추진 계획(통보용)_6_', '시나리오 정의서_지방재정(31종)_55_', 'crop2', '시나리오 정의서_지방재정(31종)_57_', '국민제안규정 전부개정령안_22_', '데이터기반행정법 시행령 제정안_5_', '221013 전국새마을지도자대회(장관님 보고용)-ver3-사진추가 (2)_1_', '임신지원 서비스 통합제공 추진계획_1_', '새마을금고법 하위법령 개정 계획(결재용)_4_', '지방공공기관 부채중점관리 세부추진 계획(통보용)_1_', '고양시 덕양구 업무매뉴얼 및 인계인수서 작성 계획_1_', '여권사실증명 무인민원발급서비스 시행 협조 요청(외교부)_3_', '개최 계획(안))_2_', '국민 제안 규정 일부개정령안 _16_', '국민 제안 규정 일부개정령안 _1_', '긴급재난지원금 사업지침(200508, 2차)_4_', '시나리오 정의서_지방재정(31종)_6_', '소방공무원의 국가직 전환 관련 제도 정비를 위한 28개 대통령령의 일부개정에 관한 대통령령안(심사의뢰)_53_', '소방공무원의 국가직 전환 관련 제도 정비를 위한 28개 대통령령의 일부개정에 관한 대통령령안(심사의뢰)_146_', '공무원제안규정 전부개정령안_1_', '지능형 초연결망 선도시범사업_공모안내서_1_', '국민이 알면 편리한 민원정보서비스  업무협약식 계획(안)_4_', '정책결정 사전점검표_29_', '소방공무원

In [58]:
pwd

'/home/selectstar'